In [6]:
import pandas as pd

In [8]:
data = pd.read_csv('./abcnews-date-text.csv',error_bad_lines=False)

C:\Program Files\KMSpico\temp\ipykernel_8744\338514806.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('./abcnews-date-text.csv',error_bad_lines=False)


In [9]:
data.columns

Index(['publish_date', 'headline_text'], dtype='object')

In [10]:
data['headline_text'].head(5)

0    aba decides against community broadcasting lic...
1       act fire witnesses must be aware of defamation
2       a g calls for infrastructure protection summit
3             air nz staff in aust strike for pay rise
4        air nz strike to affect australian travellers
Name: headline_text, dtype: object

In [14]:
data_text = data[['headline_text']] # makes it a dataframe

In [15]:
type(data_text)

pandas.core.frame.DataFrame

In [16]:
data_text

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers
...,...
1244179,two aged care residents die as state records 2...
1244180,victoria records 5;919 new cases and seven deaths
1244181,wa delays adopting new close contact definition
1244182,western ringtail possums found badly dehydrate...


In [17]:
data_text.index

RangeIndex(start=0, stop=1244184, step=1)

In [18]:
data_text['index'] = data_text.index

In [19]:
data_text

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
...,...,...
1244179,two aged care residents die as state records 2...,1244179
1244180,victoria records 5;919 new cases and seven deaths,1244180
1244181,wa delays adopting new close contact definition,1244181
1244182,western ringtail possums found badly dehydrate...,1244182


In [24]:
documents = data_text

### Data Preprocessing

In [21]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np 
np.random.seed(42)

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [37]:
def lemmatize_stemming(text):
    p = PorterStemmer()
    return p.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [38]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and \
            len(token)>3:
            result.append(lemmatize_stemming(token))
    return result

Lets see it works or not in one sample

In [39]:
doc_sample = documents[documents['index']==4310].values[0][0]

In [40]:
doc_sample

'ratepayers group wants compulsory local govt voting'

In [41]:
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)

print(words)

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


In [42]:
# after preprocessing
preprocess(doc_sample)

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

Lets now preprocess the whole documents

In [44]:
processed_docs = documents['headline_text'].map(preprocess)

#### Bag of Words on the Data Set

In [45]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [47]:
len(dictionary)

70293

In [48]:
count = 0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count >10:
        break 

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


Filter out tokens in the dictionary by their frequency.

In [49]:
dictionary.filter_extremes(no_below=15,no_above=0.5,keep_n=100000)

#### Gensim doc2bow

In [50]:
processed_docs

0                         [decid, commun, broadcast, licenc]
1                                         [wit, awar, defam]
2                     [call, infrastructur, protect, summit]
3                                [staff, aust, strike, rise]
4                       [strike, affect, australian, travel]
                                 ...                        
1244179                    [age, care, resid, state, record]
1244180               [victoria, record, case, seven, death]
1244181              [delay, adopt, close, contact, definit]
1244182    [western, ringtail, possum, badli, dehydr, hea...
1244183                  [make, close, covid, contact, rule]
Name: headline_text, Length: 1244184, dtype: object

In [51]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [52]:
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (839, 1), (3579, 1), (3580, 1)]

#### TF-IDF

In [53]:
from gensim import corpora,models

In [54]:
tfidf = models.TfidfModel(bow_corpus) #fit the model

In [55]:
type(tfidf)

gensim.models.tfidfmodel.TfidfModel

In [56]:
corpus_tfidf = tfidf[bow_corpus] # apply model to the corpus

In [63]:
corpus_tfidf[4]

[(14, 0.47235494692269364),
 (15, 0.5715156253616932),
 (16, 0.38223210042906264),
 (17, 0.5514973395100641)]

#### Running LDA using Bag of Words